### Задача 1.

Для каждого дня, представленного в таблицах `user_actions` и `courier_actions`, рассчитайте следующие показатели:

- Число новых пользователей.
- Число новых курьеров.
- Общее число пользователей на текущий день.
- Общее число курьеров на текущий день.

Колонки с показателями назовите соответственно `new_users`, `new_couriers`, `total_users`, `total_couriers`. Колонку с датами назовите `date`. Проследите за тем, чтобы показатели были выражены целыми числами. Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: `date`, `new_users`, `new_couriers`, `total_users`, `total_couriers`

```SQL
SELECT
    u.date,
    new_users,
    new_couriers,
    SUM(new_users) OVER(ORDER BY date)::INTEGER AS total_users,
    SUM(new_couriers) OVER(ORDER BY date)::INTEGER AS total_couriers
FROM
    (SELECT 
        COUNT(DISTINCT user_id) AS new_users,
        date
    FROM
    (SELECT 
        MIN(time::DATE) AS date,
        user_id
    FROM user_actions
    GROUP BY user_id) users
    GROUP BY date
    ORDER BY date) u
INNER JOIN
    (SELECT 
        COUNT(DISTINCT courier_id) AS new_couriers,
        date
    FROM
    (SELECT 
        MIN(time::DATE) AS date,
        courier_id
    FROM courier_actions
    GROUP BY courier_id) couriers
    GROUP BY date
    ORDER BY date) c
USING(date)
ORDER BY date
```

После того как составите запрос, попробуйте визуализировать результаты и постройте графики, отражающие динамику рассчитанных показателей.

Предлагаем вам построить два графика: на один поместить динамику новых пользователей и курьеров, а на второй — рост их общего числа.

![first_task_first_plot.png](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/first_task_first_plot.png)
![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/first_task_second_plot.png)

### Задача 2.

Дополните запрос из предыдущего задания и теперь для каждого дня, представленного в таблицах `user_actions` и `courier_actions`, дополнительно рассчитайте следующие показатели:

- Прирост числа новых пользователей.
- Прирост числа новых курьеров.
- Прирост общего числа пользователей.
- Прирост общего числа курьеров.

Показатели, рассчитанные на предыдущем шаге, также включите в результирующую таблицу.

Колонки с новыми показателями назовите соответственно `new_users_change`, `new_couriers_change`, `total_users_growth`, `total_couriers_growth`. Колонку с датами назовите `date`.

Все показатели прироста считайте в процентах относительно значений в предыдущий день. При расчёте показателей округляйте значения до двух знаков после запятой.

Результирующая таблица должна быть отсортирована по возрастанию даты.

Поля в результирующей таблице: `date`, `new_users`, `new_couriers`, `total_users`, `total_couriers`, `new_users_change`, `new_couriers_change`, `total_users_growth`, `total_couriers_growth`

```SQL
SELECT
    date,
    new_users,
    new_couriers,
    total_users,
    total_couriers,
    ROUND((new_users - LAG(new_users) OVER(ORDER BY date))::DECIMAL / LAG(new_users) OVER(ORDER BY date) * 100, 2) AS new_users_change,
    ROUND((new_couriers - LAG(new_couriers) OVER(ORDER BY date))::DECIMAL / LAG(new_couriers) OVER(ORDER BY date) * 100, 2) AS new_couriers_change,
    ROUND((total_users - LAG(total_users) OVER(ORDER BY date))::DECIMAL / LAG(total_users) OVER(ORDER BY date) * 100, 2) AS total_users_growth,
    ROUND((total_couriers - LAG(total_couriers) OVER(ORDER BY date))::DECIMAL / LAG(total_couriers) OVER(ORDER BY date) * 100, 2) AS total_couriers_growth
FROM (
    SELECT
        u.date AS date,
        new_users,
        new_couriers,
        SUM(new_users) OVER(ORDER BY date)::INTEGER AS total_users,
        SUM(new_couriers) OVER(ORDER BY date)::INTEGER AS total_couriers
    FROM
        (SELECT 
            COUNT(DISTINCT user_id) AS new_users,
            date
        FROM
        (SELECT 
            MIN(time::DATE) AS date,
            user_id
        FROM user_actions
        GROUP BY user_id) users
        GROUP BY date
        ORDER BY date) u
    INNER JOIN
        (SELECT 
            COUNT(DISTINCT courier_id) AS new_couriers,
            date
        FROM
        (SELECT 
            MIN(time::DATE) AS date,
            courier_id
        FROM courier_actions
        GROUP BY courier_id) couriers
        GROUP BY date
        ORDER BY date) c
    USING(date)
    ORDER BY date) f
ORDER BY date
```

После того как составите запрос, попробуйте визуализировать результаты и постройте графики, отражающие динамику рассчитанных показателей.

Предлагаем вам снова построить два графика: на один поместить приросты числа новых пользователей и курьеров, а на второй — приросты общего числа пользователей и курьеров. В данном случае для визуализации рекомендуем использовать столбиковые диаграммы (барчарты). Чтобы создать такую диаграмму, необходимо выбрать `Bar` в поле `Chart Type`.

![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/second_task_first_plot.png)
![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/second_task_second_plot.png)

### Задача 3.

Для каждого дня, представленного в таблицах `user_actions` и `courier_actions`, рассчитайте следующие показатели:

- Число платящих пользователей.
- Число активных курьеров.
- Долю платящих пользователей в общем числе пользователей на текущий день.
- Долю активных курьеров в общем числе курьеров на текущий день.

Колонки с показателями назовите соответственно `paying_users`, `active_couriers`, `paying_users_share`, `active_couriers_share`. Колонку с датами назовите `date`. Проследите за тем, чтобы абсолютные показатели были выражены целыми числами. Все показатели долей необходимо выразить в процентах. При их расчёте округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты. 

Поля в результирующей таблице: `date`, `paying_users`, `active_couriers`, `paying_users_share`, `active_couriers_share`

```SQL
SELECT
    paying.date,
    paying_users,
    active_couriers,
    ROUND(paying_users::DECIMAL / total_users * 100, 2) AS paying_users_share,
    ROUND(active_couriers::DECIMAL / total_couriers * 100, 2) AS active_couriers_share
FROM (
    SELECT
        DATE(u.date) AS date,
        paying_users,
        active_couriers
    FROM
        (SELECT 
            COUNT(DISTINCT user_id) FILTER(WHERE order_id IN 
                                           (SELECT order_id FROM courier_actions 
                                            WHERE action = 'deliver_order')) AS paying_users,
            DATE_TRUNC('day', time::DATE) AS date
        FROM user_actions
        GROUP BY date
        ORDER BY date) u
    INNER JOIN
        (SELECT 
            COUNT(DISTINCT courier_id) FILTER(WHERE order_id IN 
                                              (SELECT order_id FROM courier_actions 
                                               WHERE action = 'deliver_order')) AS active_couriers,
            DATE_TRUNC('day', time::DATE) AS date
        FROM courier_actions
        GROUP BY date
        ORDER BY date) c
    USING(date)
    ORDER BY date) paying
INNER JOIN (
    SELECT
        u.date AS date,
        SUM(new_users) OVER(ORDER BY date)::INTEGER AS total_users,
        SUM(new_couriers) OVER(ORDER BY date)::INTEGER AS total_couriers
    FROM
        (SELECT 
            COUNT(DISTINCT user_id) AS new_users,
            date
        FROM
        (SELECT 
            MIN(time::DATE) AS date,
            user_id
        FROM user_actions
        GROUP BY user_id) users
        GROUP BY date
        ORDER BY date) u
    INNER JOIN
        (SELECT 
            COUNT(DISTINCT courier_id) AS new_couriers,
            date
        FROM
        (SELECT 
            MIN(time::DATE) AS date,
            courier_id
        FROM courier_actions
        GROUP BY courier_id) couriers
        GROUP BY date
        ORDER BY date) c
    USING(date)
    ORDER BY date) total
USING(date)
```

После того как составите запрос, попробуйте визуализировать результаты и постройте графики, отражающие динамику рассчитанных показателей.

Предлагаем вам построить два графика: на один поместить динамику платящих пользователей и активных курьеров, а на второй — доли платящих пользователей и активных курьеров в их общем числе.

![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/third_task_first_plot.png)
![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/third_task_second_plot.png)

### Задача 4.

Для каждого дня, представленного в таблице `user_actions`, рассчитайте следующие показатели:

1. Долю пользователей, сделавших в этот день всего один заказ, в общем количестве платящих пользователей.
2. Долю пользователей, сделавших в этот день несколько заказов, в общем количестве платящих пользователей.

Колонки с показателями назовите соответственно `single_order_users_share`, `several_orders_users_share`. Колонку с датами назовите `date`. Все показатели с долями необходимо выразить в процентах. При расчёте долей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: `date`, `single_order_users_share`, `several_orders_users_share`

```SQL
SELECT
    date,
    ROUND(COUNT(user_id) FILTER(WHERE count_orders = 1)::DECIMAL / COUNT(user_id) * 100, 2) AS single_order_users_share,
    ROUND(COUNT(user_id) FILTER(WHERE count_orders > 1)::DECIMAL / COUNT(user_id) * 100, 2) AS several_orders_users_share
FROM
    (SELECT 
        DATE_TRUNC('day', time)::DATE AS date,
        user_id,
        COUNT(order_id) AS count_orders
    FROM user_actions
    WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    GROUP BY DATE_TRUNC('day', time), user_id) t
GROUP BY date
ORDER BY date
```

После того как составите запрос, попробуйте визуализировать результаты и постройте график, отражающий динамику рассчитанных показателей.

Поскольку в сумме оба показателя дают 100%, рекомендуем использовать столбчатую диаграмму. Чтобы столбцы были расположены друг над другом, во вкладке General в разделе Stacking можно указать параметр Stack.

![](https://github.com/akotomin/SQL/blob/main/SQL_simulator/plots/fourth_task_first_plot.png)

### * Задача 5.

Для каждого дня, представленного в таблице `user_actions`, рассчитайте следующие показатели:

1. Общее число заказов.
2. Число первых заказов (заказов, сделанных пользователями впервые).
3. Число заказов новых пользователей (заказов, сделанных пользователями в тот же день, когда они впервые воспользовались сервисом).
4. Долю первых заказов в общем числе заказов (долю п.2 в п.1).
5. Долю заказов новых пользователей в общем числе заказов (долю п.3 в п.1).

Колонки с показателями назовите соответственно `orders`, `first_orders`, `new_users_orders`, `first_orders_share`, `new_users_orders_share`. Колонку с датами назовите `date`. Проследите за тем, чтобы во всех случаях количество заказов было выражено целым числом. Все показатели с долями необходимо выразить в процентах. При расчёте долей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: `date`, `orders`, `first_orders`, `new_users_orders`, `first_orders_share`, `new_users_orders_share`

```SQL

```